In [1]:
# CONNECT
from qdrant_client import QdrantClient,models
client=QdrantClient(url="http://localhost:6333")

In [2]:
my_collection="COSINE_dieu_search_dataset"

In [3]:
#TẠO COLLECTION

#client.delete_collection(collection_name=my_collection)
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE),
)



True

In [4]:
# LOAD DATASET to COLLECTION
import json
dataset_file=open('/home/tranngochuyen/NLP/code_cty/dieu_dataset.json') #path file dataset
dataset=json.load(dataset_file)
print(dataset[0])
print(dataset[0]['vector']) 
print("Chiều dài của vector: ",len(dataset[0]['vector'])) #768
len(dataset)

{'id_van_ban': 1121, 'so-hieu': '987/QĐ-BVHTTDL', 'loai-van-ban': 'Quyết định', 'noi-ban-hanh': 'Bộ Văn hoá, Thể thao và du lịch', 'nguoi-ky': 'Tạ Quang Đông', 'ngay-ban-hanh': '11/04/2024', 'ngay-hieu-luc': '11/04/2024', 'ngay-cong-bao': 'Đang cập nhật', 'so-cong-bao': 'Đang cập nhật', 'tinh-trang': 'Còn hiệu lực', 'chuong': 0, 'muc': 0, 'dieu': 1, 'vector': [0.051355935633182526, 0.2603578269481659, -0.049113959074020386, -0.7080981135368347, -0.10140511393547058, 0.20123600959777832, -0.01681157946586609, -0.37492311000823975, 0.360429048538208, -0.434586763381958, -0.01249799132347107, 0.08201499283313751, 0.11428104341030121, -0.0427236445248127, 0.7404367923736572, -0.22520901262760162, -0.03170178458094597, -0.06216421723365784, 0.45935067534446716, 0.05647816136479378, -0.08968081325292587, 0.33140677213668823, 0.02343665435910225, 0.7951955795288086, -0.3621683120727539, -0.06153327599167824, -0.5268372297286987, -0.008691098541021347, 0.056755173951387405, -0.1568821072578430

989

In [5]:
client.upload_points(
    collection_name=my_collection,
    points=[
        models.PointStruct(
            id=idx,vector=doc['vector'],payload=doc
        )for idx,doc in enumerate(dataset)
    ],
)

SEARCH

In [6]:
# search toàn bộ văn bản
def search_full(query_vector, limit):
   my_collection="search_dataset"
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      limit=limit
   )
   for i in search_result:
      print(i)


# thêm filter
def search_with_filter(query_vector, limit):
   my_collection="search_dataset"
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      query_filter=models.Filter(
         must=[models.FieldCondition(key="dieu", match=models.MatchValue(value="2"))]
      ),
      limit=limit
   )

 
   return search_result

# chỉ in ra 1 số thông tin cần thiết
# import json
# import pandas as pd
# df=pd.read_csv('data.csv')
# data_csv=df.to_numpy()
#print(data) #(71727, 14)


def search_with_some_inf(query_vector, limit):
   #global data_csv
   my_collection="COSINE_dieu_search_dataset"
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      limit=limit,
      search_params=models.SearchParams(
               exact=True,  # Turns on the exact search mode KNN
            ),
   )
   #print question
   if len(search_result)==0:
      print("Không có thông tin tìm kiếm")
   else:
      print("Kết quả thông tin tìm kiếm là:")
      answer_array=[]
      for i in search_result:

         answer_json={
            'index':i.id,
            'score':i.score,
            #'noi dung':data_csv[i.id,-1],
            'id_van_ban':i.payload['id_van_ban'],
            'so-hieu':i.payload['so-hieu'],
            'loai-van-ban':i.payload['loai-van-ban'],
            'noi-ban-hanh':i.payload['noi-ban-hanh'],
            'nguoi-ky':i.payload['nguoi-ky'],
            'ngay-ban-hanh':i.payload['ngay-ban-hanh'],
            'so-cong-bao':i.payload['so-cong-bao'],
            'tinh-trang':i.payload['tinh-trang'],
            'chuong':i.payload['chuong'],
            'muc':i.payload['muc'],
            'dieu':i.payload['dieu'],
            
            
         }
         if len(answer_array)==0:
            answer_array.append(answer_json)
         else:
            count=0
            for answer in answer_array:
            #id là của 1 văn bản , phải thêm chương, điều, mục
               if (answer_json['index']==answer['index']):
                  count=count+1
            if count==0:
               answer_array.append(answer_json)

     
      
      for i in answer_array:
         print(i)



In [7]:
# THỬ  VỚI 1 QUERY VECTOR CÓ SẴN TRONG DATASET
query_vector=dataset[1]
print(query_vector)
#print(data_csv[10-1])
#print(type(query_vector))
print("\nResult:")
search_with_some_inf(query_vector['vector'],10)


{'id_van_ban': 1121, 'so-hieu': '987/QĐ-BVHTTDL', 'loai-van-ban': 'Quyết định', 'noi-ban-hanh': 'Bộ Văn hoá, Thể thao và du lịch', 'nguoi-ky': 'Tạ Quang Đông', 'ngay-ban-hanh': '11/04/2024', 'ngay-hieu-luc': '11/04/2024', 'ngay-cong-bao': 'Đang cập nhật', 'so-cong-bao': 'Đang cập nhật', 'tinh-trang': 'Còn hiệu lực', 'chuong': 0, 'muc': 0, 'dieu': 2, 'vector': [-0.5465320348739624, 0.8042479753494263, 0.003046305850148201, 0.31441229581832886, -0.06521207839250565, -0.4376092553138733, -0.7156124711036682, 0.8796327710151672, -0.15527796745300293, -0.2156277298927307, -0.22685390710830688, 0.5675731301307678, -0.025976501405239105, 1.0139718055725098, -0.08208893239498138, -0.06776393204927444, 2.5987942218780518, 2.3257555961608887, -0.34825703501701355, 2.0681068897247314, 0.6790419816970825, -0.05174092575907707, -0.007456663995981216, 0.8807516098022461, 0.7226629257202148, 0.7138738632202148, -0.8179367780685425, -0.03988868370652199, 0.04398626461625099, -0.22653478384017944, 0.14

In [8]:

import torch
from vector_dieu_cau_hoi import text_to_vector
text = '''Thủ tục hành chính lĩnh vực Thi đua, khen thưởng có số thứ tự 01, 02 điểm A1 mục A danh mục 1 ban hành kèm theo Quyết định số 786/QĐ-BVHTTDL ngày 31 tháng 3 năm 2023 của Bộ trưởng Bộ Văn hóa, Thể thao và Du lịch về việc công bố thủ tục hành chính nội bộ giữa các cơ quan, đơn vị trực thuộc Bộ và trong nội bộ cơ quan, đơn vị trực thuộc Bộ thuộc phạm vi chức năng quản lý của Bộ Văn hóa, Thể thao và Du lịch hết hiệu lực thi hành kể từ ngày Quyết định này có hiệu lực thi hành'''
query_vector=text_to_vector(text)[0].tolist() #<class 'numpy.ndarray'>  => list

#print(query_vector.shape) #(2, 768)
#print(len(query_vector))

# search mỗi câu với 1 kết quả tương ứng

print(f"Top Result: ")
answer_array=search_with_some_inf(query_vector,10)

'''
1. Sở Khoa học và Công nghệ:

- Chủ trì, phối hợp với các cơ quan có liên quan, xây dựng mới quy trình nội bộ giải quyết thủ tục hành chính công bố tại Điều 1 Quyết định này, trình Chủ tịch UBND tỉnh phê duyệt theo quy định.

- Phối hợp với Sở Thông tin và Truyền thông cập nhật, đăng tải công khai kịp thời, đầy đủ, chính xác Danh mục thủ tục hành chính, bộ phận tạo thành của thủ tục hành chính công bố tại Điều 1 Quyết định này trên Cổng Dịch vụ công, Hệ thống thông tin giải quyết thủ tục hành chính của tỉnh theo quy định.

2. Văn phòng UBND tỉnh cập nhật, đăng tải công khai kịp thời, đầy đủ, chính xác các dữ liệu thủ tục hành chính vào Cơ sở dữ liệu quốc gia theo quy định.'''

/home/tranngochuyen/anaconda3/envs/search_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tranngochuyen/anaconda3/envs/search_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Đang xử lý line =  Thủ tục hành chính lĩnh vực Thi đua, khen thưởng có số thứ tự 01, 02 điểm A1 mục A danh mục 1 ban hành kèm theo Quyết định số 786/QĐ-BVHTTDL ngày 31 tháng 3 năm 2023 của Bộ trưởng Bộ Văn hóa, Thể thao và Du lịch về việc công bố thủ tục hành chính nội bộ giữa các cơ quan, đơn vị trực thuộc Bộ và trong nội bộ cơ quan, đơn vị trực thuộc Bộ thuộc phạm vi chức năng quản lý của Bộ Văn hóa, Thể thao và Du lịch hết hiệu lực thi hành kể từ ngày Quyết định này có hiệu lực thi hành
Top Result: 
Kết quả thông tin tìm kiếm là:
{'index': 664, 'score': 0.102350384, 'id_van_ban': 1202, 'so-hieu': '584/QĐ-UBND', 'loai-van-ban': 'Quyết định', 'noi-ban-hanh': 'Tỉnh Phú Thọ', 'nguoi-ky': 'Bùi Văn Quang', 'ngay-ban-hanh': '29/03/2024', 'so-cong-bao': 'Đang cập nhật', 'tinh-trang': 'Còn hiệu lực', 'chuong': 0, 'muc': 0, 'dieu': 2}
{'index': 650, 'score': 0.100197054, 'id_van_ban': 1195, 'so-hieu': '220/QĐ-BXD', 'loai-van-ban': 'Quyết định', 'noi-ban-hanh': 'Bộ Xây dựng', 'nguoi-ky': 'Nguy